In [1]:
import sys
print(sys.executable)
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2
from PIL import Image
from io import BytesIO
from sklearn.model_selection import ShuffleSplit




c:\users\zhizh\appdata\local\conda\conda\envs\tf-gpu\python.exe


In [ ]:
vid = os.listdir('set_vid')
count = 0
import ffmpeg
#for v in vid:
    #os.rename('set_vid/' + v, 'set_vid/' + v[0:9] + str(count) + '.h264') 
    #count += 1
    #os.system("ffmpeg -i set_vid/" + v + " set_vid/" + v[0:-5] + ".mp4")

In [2]:
# im = cv2.imread("set2/2w.jpg")
# im = cv2.resize(im,(28,28))
# cv2.imwrite("resized-2w.jpg", im)

In [46]:
# save images as numpy array
directory_list = ['set3','set2','set1']
directory_list2 = ['set3','set2','set1']
train_array = []
train_label = []
eval_array = []
eval_label = []

'''
0 - w
1 - a
2 - s
3 - d
4 - stop
[1,0,0,0]
'''

def return_label(inp):
    if (inp == 'w'):
        return np.array([1.0,0,0,0,0])
        #return np.array([0])
    elif (inp == 'a'):
        return np.array([0,1.0,0,0,0])
        #return np.array([1])
    elif (inp == 's'):
        return np.array([0,0,1.0,0,0])
        #return np.array([2])
    elif (inp == 'd'):
        return np.array([0,0,0,1.0,0])
        #return np.array([3])
    elif (inp == 'x'):
        return np.array([0,0,0,0,1.0])
        #return np.array([4])

for d in directory_list:
    img_list = os.listdir(d)
    for i in img_list:
        im = cv2.imread(d + "/" + i)
        im = cv2.resize(im,(32,32))
        train_array.append(im)
        train_label.append(return_label(i[i.find('.')-1]))

        
        
for d in directory_list2:
    img_list = os.listdir(d)
    for i in img_list:
        im = cv2.imread(d + "/" + i)
        im = cv2.resize(im,(32,32))
        eval_array.append(im)
        eval_label.append(return_label(i[i.find('.')-1]))
directory_list3 = ['/stop','/straight','/turn_left','/turn_right']

count = 0
for d in directory_list3:
    img_list = os.listdir('set_vid'+d)
    for i in img_list:
        im = cv2.imread('set_vid' + d + '/' + i)
        im = cv2.resize(im,(32,32))
        train_array.append(im)
        if (count == 0):
            train_label.append(np.array([0,0,0,0,1.0]))
        elif (count == 1):
            train_label.append(np.array([1.0,0,0,0,0]))
        elif (count == 2):
            train_label.append(np.array([0,1.0,0,0,0]))
        elif (count == 3):
            train_label.append(np.array([0,0,0,1.0,0]))
        
    count += 1

In [47]:
LABELS = 5 # Number of different types of labels (1-10)
WIDTH = 32 # width / height of the image
CHANNELS = 3 # Number of colors in the image (greyscale)

VALID = 30 # Validation data size

STEPS = 20000 #20000   # Number of steps to run
BATCH = 100 # Stochastic Gradient Descent batch size
PATCH = 5 # Convolutional Kernel size
DEPTH = 12 #32 # Convolutional Kernel depth size == Number of Convolutional Kernels
HIDDEN = 100 #1024 # Number of hidden neurons in the fully connected layer

LR = 0.001 # Learning rate

tf_data = tf.placeholder(tf.float32, shape=(None, WIDTH, WIDTH, CHANNELS))
tf_labels = tf.placeholder(tf.float32, shape=(None, LABELS))

In [48]:
w1 = tf.Variable(tf.truncated_normal([PATCH, PATCH, CHANNELS, DEPTH], stddev=0.1))
b1 = tf.Variable(tf.zeros([DEPTH]))
w2 = tf.Variable(tf.truncated_normal([PATCH, PATCH, DEPTH, 2*DEPTH], stddev=0.1))
b2 = tf.Variable(tf.constant(1.0, shape=[2*DEPTH]))
w3 = tf.Variable(tf.truncated_normal([WIDTH // 4 * WIDTH // 4 * 2*DEPTH, HIDDEN], stddev=0.1))
b3 = tf.Variable(tf.constant(1.0, shape=[HIDDEN]))
w4 = tf.Variable(tf.truncated_normal([HIDDEN, LABELS], stddev=0.1))
b4 = tf.Variable(tf.constant(1.0, shape=[LABELS]))

def logits(data):
    # Convolutional layer 1
    x = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
    x = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    x = tf.nn.relu(x + b1)
    # Convolutional layer 2
    x = tf.nn.conv2d(x, w2, [1, 1, 1, 1], padding='SAME')
    x = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    x = tf.nn.relu(x + b2)
    # Fully connected layer
    x = tf.reshape(x, (-1, WIDTH // 4 * WIDTH // 4 * 2*DEPTH))
    x = tf.nn.relu(tf.matmul(x, w3) + b3)
    return tf.matmul(x, w4) + b4

# Prediction:
tf_pred = tf.nn.softmax(logits(tf_data))

In [49]:
tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits(tf_data), 
                                                                 labels=tf_labels))
tf_acc = 100*tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(tf_pred, 1), tf.argmax(tf_labels, 1))))

#tf_opt = tf.train.GradientDescentOptimizer(LR)
#tf_opt = tf.train.AdamOptimizer(LR)
tf_opt = tf.train.RMSPropOptimizer(LR)
tf_step = tf_opt.minimize(tf_loss)

In [51]:
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

train_data = np.asarray(train_array, dtype=np.float32)
train_labels = np.asarray(train_label, dtype=np.int32)

valid_data = np.asarray(eval_array, dtype=np.float32)
valid_labels = np.asarray(eval_label, dtype=np.int32)


#indices = np.random.permutation(data.shape[0])
#train_data, valid_data = data[:int(len(indices)*.8)],data[int(len(indices)*.8):]

#indices2 = np.random.permutation(labels.shape[0])
#train_labels, valid_labels = labels[:int(len(indices)*.8)],labels[int(len(indices)*.8):]


In [52]:
ss = ShuffleSplit(n_splits=STEPS, train_size=BATCH)
ss.get_n_splits(train_data, train_labels)
history = [(0, np.nan, 10)] # Initial Error Measures
for step, (idx, _) in enumerate(ss.split(train_data,train_labels), start=1):
    fd = {tf_data:train_data[idx], tf_labels:train_labels[idx]}
    session.run(tf_step, feed_dict=fd)
    if step%500 == 0:
        fd = {tf_data:valid_data, tf_labels:valid_labels}
        valid_loss, valid_accuracy = session.run([tf_loss, tf_acc], feed_dict=fd)
        history.append((step, valid_loss, valid_accuracy))
        print('Step %i \t Valid. Acc. = %f'%(step, valid_accuracy), end='\n')
    

Step 500 	 Valid. Acc. = 61.395348
Step 1000 	 Valid. Acc. = 84.031006
Step 1500 	 Valid. Acc. = 91.782944
Step 2000 	 Valid. Acc. = 97.984497
Step 2500 	 Valid. Acc. = 99.069771
Step 3000 	 Valid. Acc. = 97.829453
Step 3500 	 Valid. Acc. = 91.937981
Step 4000 	 Valid. Acc. = 97.519379
Step 4500 	 Valid. Acc. = 93.798447
Step 5000 	 Valid. Acc. = 97.054260
Step 5500 	 Valid. Acc. = 97.519379
Step 6000 	 Valid. Acc. = 98.139534
Step 6500 	 Valid. Acc. = 97.674416
Step 7000 	 Valid. Acc. = 98.759689
Step 7500 	 Valid. Acc. = 94.883720
Step 8000 	 Valid. Acc. = 99.069771
Step 8500 	 Valid. Acc. = 99.689926
Step 9000 	 Valid. Acc. = 98.139534
Step 9500 	 Valid. Acc. = 99.534882
Step 10000 	 Valid. Acc. = 99.689926
Step 10500 	 Valid. Acc. = 99.844963
Step 11000 	 Valid. Acc. = 98.604652
Step 11500 	 Valid. Acc. = 99.534882
Step 12000 	 Valid. Acc. = 99.379845
Step 12500 	 Valid. Acc. = 96.744186
Step 13000 	 Valid. Acc. = 99.224808
Step 13500 	 Valid. Acc. = 99.379845
Step 14000 	 Valid. A

In [53]:
saver = tf.train.Saver()
saver.save(session, os.path.abspath(os.path.join(os.getcwd(), 'model_2')))

'D:\\Projectx\\makeathon2019\\model_2'